In [1]:
# import sys
# !{sys.executable} -m pip install ing_theme_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
# import qbstyles
# from ing_theme_matplotlib import mpl_style
import glacierml as gl
from tqdm import tqdm

# Step 1: Organize the data

## A. Load and organize the Glathida dataset

In [14]:
# raw_dataset = pd.read_csv('/data/fast0/datasets/glathida-3.1.0/data/T.csv')
T = pd.read_csv('/home/sa42/data/glac/T_models/T.csv')
T = T.drop([
#     'GlaThiDa_ID',
    'POLITICAL_UNIT',
    'GLACIER_NAME',
    'GLACIER_DB',
    'GLACIER_ID',
    'SURVEY_DATE',
    'ELEVATION_DATE',
    'MEAN_THICKNESS',
    'MEAN_THICKNESS_UNCERTAINTY',
    'MAXIMUM_THICKNESS',
    'MAX_THICKNESS_UNCERTAINTY',
    'SURVEY_METHOD',
    'SURVEY_METHOD_DETAILS',
    'NUMBER_OF_SURVEY_POINTS',
    'NUMBER_OF_SURVEY_PROFILES',
    'TOTAL_LENGTH_OF_SURVEY_PROFILES',
    'INTERPOLATION_METHOD',
    'INVESTIGATOR',
    'SPONSORING_AGENCY',
    'REFERENCES',
    'DATA_FLAG',
    'REMARKS'
],axis=1)
T = T.dropna()


,GlaThiDa_ID,LAT,LON,AREA,MEAN_SLOPE
32,33,48.75000,-121.83000,2.900,0.0
33,34,58.38000,-134.36000,11.700,0.0
34,35,50.06134,87.69741,5.150,0.0
35,36,50.04967,87.74552,2.840,0.0
36,37,50.08008,87.69025,5.950,0.0
...,...,...,...,...,...
1004,2495,77.57700,23.46980,186.971,7.0
1005,2496,77.81080,23.71840,581.764,4.0
1006,2497,78.92550,18.34900,876.603,6.0
1007,2498,78.86400,19.82440,594.902,4.0


In [17]:
TTT = pd.read_csv('/home/sa42/data/glac/T_models/TTT.csv')
TTT = TTT.drop([
#     'GlaThiDa_ID',
    'POLITICAL_UNIT',
    'GLACIER_NAME',
    'SURVEY_DATE',
    'PROFILE_ID',
    'POINT_ID',
#     'POINT_LAT',
#     'POINT_LON',
#     'ELEVATION',
#     'THICKNESS',
    'THICKNESS_UNCERTAINTY',
    'DATA_FLAG',
    'REMARKS'
],axis=1)


/home/sa42/miniconda3/envs/python-cartopy-f/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,4,5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,GlaThiDa_ID,POINT_LAT,POINT_LON,ELEVATION,THICKNESS
0,33,48.767380,-121.819644,2962.0,0
1,33,48.764904,-121.821909,2813.0,29
2,33,48.761662,-121.825264,2598.0,41
3,33,48.757063,-121.829107,2383.0,71
4,33,48.753715,-121.832006,2284.0,82
...,...,...,...,...,...
3854274,6630,47.092690,12.380504,3329.0,36
3854275,6630,47.093780,12.379642,3353.0,26
3854276,6630,47.094839,12.378200,3381.0,39
3854277,6630,47.094829,12.378174,3381.0,39


In [18]:
df = pd.merge(TTT,T, how='inner',on='GlaThiDa_ID')
df = df.drop([
    'GlaThiDa_ID'
],axis=1)


,POINT_LAT,POINT_LON,ELEVATION,THICKNESS,LAT,LON,AREA,MEAN_SLOPE
0,48.767380,-121.819644,2962.0,0,48.750,-121.8300,2.900,0.0
1,48.764904,-121.821909,2813.0,29,48.750,-121.8300,2.900,0.0
2,48.761662,-121.825264,2598.0,41,48.750,-121.8300,2.900,0.0
3,48.757063,-121.829107,2383.0,71,48.750,-121.8300,2.900,0.0
4,48.753715,-121.832006,2284.0,82,48.750,-121.8300,2.900,0.0
...,...,...,...,...,...,...,...,...
906732,79.148849,19.458450,111.0,105,79.115,19.4916,76.893,7.0
906733,79.150085,19.461258,103.0,101,79.115,19.4916,76.893,7.0
906734,79.151077,19.463531,94.0,93,79.115,19.4916,76.893,7.0
906735,79.152054,19.465820,80.0,100,79.115,19.4916,76.893,7.0


### separate out features - what will be trained to predict desired attribute

In [19]:
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

#define label - attribute training to be picked
train_labels = train_features.pop('THICKNESS')
test_labels = test_features.pop('THICKNESS')

train_features.describe().transpose()[['mean', 'std']]

,mean,std
POINT_LAT,77.301107,7.705735
POINT_LON,17.750111,11.898922
ELEVATION,634.567862,797.298420
LAT,77.303030,7.706165
LON,17.780224,11.883406
AREA,288.418078,346.939971
MEAN_SLOPE,9.231594,3.642620


# Step 2: Setup the Model

### Establish normalization layers

In [20]:
normalizer = {}
variable_list = list(train_features)
for variable_name in tqdm(variable_list):

    normalizer[variable_name] = gl.preprocessing.Normalization(input_shape=[1,], axis=None)
    normalizer[variable_name].adapt(np.array(train_features[variable_name]))
    
    
normalizer['ALL'] = gl.preprocessing.Normalization(axis=-1)
normalizer['ALL'].adapt(np.array(train_features))

  0%|          | 0/7 [00:00<?, ?it/s]2022-03-23 16:44:37.388337: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-23 16:44:37.388629: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sermeq.ess.washington.edu): /proc/driver/nvidia/version does not exist
2022-03-23 16:44:37.389887: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 7/7 [00:59<00:00,  8.52s/it]


In [21]:
linear_model = {}
linear_history = {}
linear_results = {}

# Linear regression

In [ ]:
%%time
for variable_name in tqdm(variable_list):

    linear_model[variable_name] = gl.build_linear_model(normalizer[variable_name])
    linear_history[variable_name] = linear_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=100,
                                        verbose=0,
                                        validation_split = 0.2)
    
    
    linear_results[variable_name] = linear_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# fig,ax=plt.subplots(2,6,figsize=(10,10))
# for i, variable_name in enumerate(variable_list):
# #     mpl_style("dark")
#     ax = plt.subplot(4,4,i+1)
#     plt.subplot(4,4,i+1)
#     gl.plot_loss(linear_history[variable_name])
#     ax.set_title(variable_name)
# #     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_loss.eps")

# Linear regression with multiple inputs

In [ ]:
linear_model = gl.build_linear_model(normalizer['ALL'])

linear_history['MULTI'] = linear_model.fit(
train_features, train_labels,        
   epochs=100,
   verbose=0,
   validation_split = 0.2)

linear_results['MULTI'] = linear_model.evaluate(
    test_features,
    test_labels, verbose=0)

In [ ]:
# gl.plot_loss(history_full)
# # plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_full_loss.eps")

# DNN regression

## Define regression functions

In [ ]:
dnn_model = {}
dnn_history = {}
dnn_results = {}

In [ ]:
for variable_name in tqdm(variable_list):

    dnn_model[variable_name] = gl.build_dnn_model(normalizer[variable_name])
    dnn_history[variable_name] = dnn_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=100,
                                        verbose=0,
                                        validation_split = 0.2)
    dnn_results[variable_name] = dnn_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

In [ ]:
# fig,ax=plt.subplots(4,4,figsize=(10,10))
# for i, variable_name in enumerate(variable_list):
#     plt.subplot(4,4,i+1)
#     xmax = np.max(train_features[variable_name])
#     xmin = np.min(train_features[variable_name])
#     x = tf.linspace(xmin, xmax, 101)
#     y = dnn_model[variable_name].predict(x)
#     plot_single_model_variable(x,y,variable_name)

In [ ]:
# fig,ax=plt.subplots(4,4,figsize=(10,10))
# fig.patch.set_facecolor('w')
# for i, variable_name in enumerate(variable_list):
#     ax = plt.subplot(4,4,i+1)
#     gl.plot_loss(dnn_history[variable_name])
#     ax.set_ylim([13,22])
#     ax.set_title(variable_name)
# #     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_loss.eps")
# plt.tight_layout()

# Full model

In [ ]:
dnn_full_model = gl.build_dnn_model(normalizer['ALL'])

dnn_history['MULTI'] = dnn_full_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

dnn_results['MULTI'] = dnn_full_model.evaluate(
    test_features,
    test_labels, verbose=0)

In [ ]:
gl.plot_loss(dnn_history['MULTI'])
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_full_loss.png")

In [ ]:
# y = dnn_full_model.predict(test_features)
# plt.plot(test_labels,y,'.')
# plt.plot((0,200),(0,200),'-')
# plt.xlabel('True Thickness (m)')
# plt.ylabel('Model Thickness (m)')
# plt.xlim((0,200))
# plt.ylim((0,200))
# # plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_res.EPS")

# Collect Results

In [ ]:
dfs = pd.DataFrame()
for variable_name in list(dnn_history):    
    df1 = pd.DataFrame(dnn_history[variable_name].history)
    df1 = df1.loc[[df1.last_valid_index()]]
    df1['Architecture'] = 'DNN'
    df1.insert(0, 'Variable', [variable_name])

    df2 = pd.DataFrame(linear_history[variable_name].history)
    df2 = df2.loc[[df2.last_valid_index()]]
    df2['Architecture'] = 'Linear'
    df2.insert(0, 'Variable', [variable_name])
    
    df = pd.concat([df1,df2])
    dfs = dfs.append(df)
    
df = dfs[[
    'Architecture',
    'Variable',
    'loss',
    'val_loss'
]]
df.rename(columns = {
    'loss':'Training Loss',
    'val_loss':'Test loss'
},inplace=True)
df = df.sort_values(by=['Architecture','Variable'], ascending=[False,False])
print(df.to_latex(index=False))